In [1]:
using Pipe
using Iterators
#using HDF5, JLD

In [2]:
using PyCall
import PyCall.@pysym
@pyimport warnings
warnings.filterwarnings("ignore")

In [4]:
@pyimport nltk.corpus as nltk_corpus

done (generic function with 73 methods)

In [6]:
function cnf!(x::PyObject)
    x[:collapse_unary]()
    x[:chomsky_normal_form]()
    
    x
end

cnf! (generic function with 1 method)

In [7]:

function tupup(x::Any)
    x
end

function tupup(tree::PyObject)
    if pyisinstance(tree, @pysym :PyList_Type)
        if tree[:__len__]()==0
            nothing
        elseif tree[:__len__]()==1
            tupup(pyeval("tree[0]",PyObject; tree=tree))
        else
            @assert tree[:__len__]()==2
            left = tupup(pyeval("tree[0]",PyObject; tree=tree);)
            right = tupup(pyeval("tree[1]",PyObject; tree=tree);)
            if left!=nothing &&  right!=nothing
                (tupup(left), tupup(right))
            elseif left==nothing
                tupup(right)
            elseif right==nothing
                tupup(left)
            end
            
            
        end
    else
        convert(String,tree)
    end
end
    
function tupup(x::Nothing)
    error("Can't Make tuple of nothing")
end

tupup (generic function with 3 methods)

In [8]:
function loadtrees(reader::PyObject)
    @pipe (
    pycall(reader["parsed_sents"], PyObject) 
    |> PyObjectIter
    |> imap(cnf!,_)
    |> imap(tupup,_) 
    |> filter(x->x!==nothing,_) 
    |> collect)
end

loadtrees (generic function with 1 method)

In [ ]:
open("training_sents.jsz","w") do fs
    serialize(fs, loadtrees(nltk_corpus.treebank))
end

In [12]:
@pyimport glob
opinosis_filenames = glob.glob("corpora/Opinosis/processed/topics/*.parsed.txt");
opinosis_reader = nltk_corpus.BracketParseCorpusReader(".", opinosis_filenames, comment_char="S")
#Comment character is `S` because the Standford Parser outputs "Sentence Skipped" and "SENTENCE_SKIPPED_OR_UNPARASABLE" as log messages

open("opinosis_sents.jsz","w") do fs
    serialize(fs, loadtrees(opinosis_reader))
end

In [13]:
 loadtrees(opinosis_reader)

6314-element Array{Any,1}:
 (("Parking",("was","expensive")),("but",(("I",("think",("this",("is",("common",("for",("San","Fran"))))))),".")))                                                                                                                                                                                                                                                                                               
 ("there",(("is",(("a","fee"),(("for","parking"),(("but","well"),("worth",("it",("seeing",("no",("where",("to",("park",("if",("you",("do",("have",("a","car")))))))))))))))),"."))                                                                                                                                                                                                                               
 ((("The","parking"),("was",("free",(",",("which",("was","great")))))),(",",("and",((("the","hotel"),("was",("conveniently",("located",("for",(("public","transport"),(",

In [ ]:
questionbank_reader = nltk_corpus.BracketParseCorpusReader(".", ["../Corpus/corpora/QuestionBank/4000qs.txt"])

open("questionbank_sents.jsz","w") do fs
    serialize(fs, loadtrees(questionbank_reader))
end